In [3]:
from headers import *

In [4]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

In [5]:
df = pd.read_excel("input_df.xlsx")
df.index.name = "index"
df.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice,commuter,age_50_65,age_65
index,,,,,,,,,,,,
0,1,1,1,0,20,0,20,10,0,0,0,1
1,1,2,1,0,20,0,20,5,1,0,0,1
2,1,3,1,20,0,20,0,10,0,0,0,1
3,1,4,1,20,0,20,0,5,1,0,0,1
4,1,5,1,20,0,0,20,10,0,0,0,1
5,1,6,1,20,0,0,20,5,1,0,0,1
6,2,1,1,0,15,0,15,10,0,1,0,0
7,2,2,1,0,15,0,15,5,0,1,0,0
8,2,3,1,15,0,15,0,10,0,1,0,0


In [6]:
df['B_cr4first'] = [1 if x > 0 else 0 for x in df['now_ivt_4']]
df['B_cr3first'] = [1 if x > 0 else 0 for x in df['now_ivt_3']]
df['B_cr3second'] = [1 if x > 0 else 0 for x in df['wait_ivt_3']]
df['B_cr2second'] = [1 if x > 0 else 0 for x in df['wait_ivt_2']]
df['IVT_time'] = df[['now_ivt_4','now_ivt_3']].max(axis=1)
df_used = df[['pass_ID','scenario','time_crit','B_cr4first','B_cr3first','B_cr3second','B_cr2second','IVT_time','WT_time','output_choice']]
df_used.head(20)

,pass_ID,scenario,time_crit,B_cr4first,B_cr3first,B_cr3second,B_cr2second,IVT_time,WT_time,output_choice
index,,,,,,,,,,
0,1,1,1,0,1,0,1,20,10,0
1,1,2,1,0,1,0,1,20,5,1
2,1,3,1,1,0,1,0,20,10,0
3,1,4,1,1,0,1,0,20,5,1
4,1,5,1,1,0,0,1,20,10,0
5,1,6,1,1,0,0,1,20,5,1
6,2,1,1,0,1,0,1,15,10,0
7,2,2,1,0,1,0,1,15,5,0
8,2,3,1,1,0,1,0,15,10,0


# models 01x - general framework
### (analogous to (Preston et al., 2017))
### a) ASC / no ASC
### b) general / extended / time-critical / non-time-critical
### = 8 models in total

## Model 011 - general model with ASC
### dep1 = 0
### dep2 = [ASC_second] + [B_timecrit] * s1 + [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt

In [5]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [6]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
ASC_SECOND = Beta('ASC_SECOND',0,None,None,0)
BETA_timecrit = Beta('BETA_timecrit',0,None,None,0)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s1 = time_crit 
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = ASC_SECOND + \
     BETA_timecrit * s1 + \
     BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [7]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_011_general_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_SECOND=	0.754
BETA_CR2_second=	0.414
BETA_CR3_first=	-0.678
BETA_CR3_second=	0.34
BETA_CR4_first=	1.43
BETA_IVT=	0.0278
BETA_WT=	-0.275
BETA_timecrit=	-1.56
                    Value   Std err     t-test       p-value  Rob. Std err  \
ASC_SECOND       0.753754  0.095575   7.886519  3.108624e-15      0.097489   
BETA_CR2_second  0.414201  0.069792   5.934763  2.942705e-09      0.070205   
BETA_CR3_first  -0.677508  0.082887  -8.173895  2.220446e-16      0.083688   
BETA_CR3_second  0.339554  0.080529   4.216515  2.481063e-05      0.080872   
BETA_CR4_first   1.431263  0.081219  17.622196  0.000000e+00      0.081768   
BETA_IVT         0.027803  0.004338   6.409666  1.458387e-10      0.004869   
BETA_WT         -0.275226  0.020846 -13.203138  0.000000e+00      0.020810   
BETA_timecrit   -1.561388  0.105993 -14.731109  0.000000e+00      0.106166   

                 Rob. t-test  Rob. p-value  
ASC_SECOND          7.731661  1.065814e-14  
BETA_CR2_second     5.899871  3.637857e-09  
B

## Model 012 - general model
### dep1 = 0
### dep2 = [B_timecrit] * s1 + [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt

In [8]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_timecrit = Beta('BETA_timecrit',0,None,None,0)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s1 = time_crit 
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = BETA_timecrit * s1 + \
     BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [9]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_012_general"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_CR2_second=	0.791
BETA_CR3_first=	-0.301
BETA_CR3_second=	0.716
BETA_CR4_first=	1.81
BETA_IVT=	0.0278
BETA_WT=	-0.275
BETA_timecrit=	-1.56
                    Value   Std err     t-test       p-value  Rob. Std err  \
BETA_CR2_second  0.791074  0.104474   7.571952  3.685940e-14      0.106071   
BETA_CR3_first  -0.300629  0.117720  -2.553768  1.065643e-02      0.119544   
BETA_CR3_second  0.716421  0.118924   6.024194  1.699546e-09      0.120308   
BETA_CR4_first   1.808124  0.115372  15.672170  0.000000e+00      0.116855   
BETA_IVT         0.027803  0.004338   6.409679  1.458258e-10      0.004869   
BETA_WT         -0.275223  0.020845 -13.203030  0.000000e+00      0.020809   
BETA_timecrit   -1.561396  0.105992 -14.731200  0.000000e+00      0.106165   

                 Rob. t-test  Rob. p-value  
BETA_CR2_second     7.457956  8.792966e-14  
BETA_CR3_first     -2.514805  1.190981e-02  
BETA_CR3_second     5.954892  2.602434e-09  
BETA_CR4_first     15.473284  0.000000e+00  
BETA_I

## Model 013 general model, time-critical trips only, ASC
### dep1 = 0
### dep2 = [ASC_second] + [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt

In [10]:
# filter records
df_used_02 = df_used[df_used['time_crit']==1]

In [11]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used_02)

In [12]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
ASC_SECOND = Beta('ASC_SECOND',0,None,None,0)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = ASC_SECOND + \
     BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [13]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_013_general_timecrit_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_SECOND=	0.129
BETA_CR2_second=	0.0924
BETA_CR3_first=	-0.942
BETA_CR3_second=	0.0363
BETA_CR4_first=	1.07
BETA_IVT=	0.0225
BETA_WT=	-0.297
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_SECOND       0.128755  0.132867  0.969050  3.325201e-01      0.130536   
BETA_CR2_second  0.092434  0.098621  0.937261  3.486243e-01      0.097986   
BETA_CR3_first  -0.941562  0.141551 -6.651771  2.895861e-11      0.139627   
BETA_CR3_second  0.036321  0.114598  0.316945  7.512854e-01      0.114031   
BETA_CR4_first   1.070317  0.116646  9.175797  0.000000e+00      0.115964   
BETA_IVT         0.022522  0.005813  3.874399  1.068883e-04      0.005752   
BETA_WT         -0.297456  0.031772 -9.362128  0.000000e+00      0.031943   

                 Rob. t-test  Rob. p-value  
ASC_SECOND          0.986360  3.239565e-01  
BETA_CR2_second     0.943334  3.455099e-01  
BETA_CR3_first     -6.743397  1.547251e-11  
BETA_CR3_second     0.318521  7.500898e-01  
BETA_CR4_first 

## Model 014 general model, time-critical trips only
### dep1 = 0
### dep2 = [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt

In [14]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [15]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_014_general_timecrit"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_CR2_second=	0.157
BETA_CR3_first=	-0.877
BETA_CR3_second=	0.101
BETA_CR4_first=	1.13
BETA_IVT=	0.0225
BETA_WT=	-0.297
                    Value   Std err    t-test       p-value  Rob. Std err  \
BETA_CR2_second  0.156813  0.145819  1.075394  2.821984e-01      0.143806   
BETA_CR3_first  -0.877189  0.191029 -4.591913  4.392013e-06      0.187882   
BETA_CR3_second  0.100696  0.167560  0.600956  5.478692e-01      0.165782   
BETA_CR4_first   1.134697  0.153725  7.381350  1.567635e-13      0.152326   
BETA_IVT         0.022522  0.005813  3.874385  1.068944e-04      0.005752   
BETA_WT         -0.297456  0.031772 -9.362125  0.000000e+00      0.031943   

                 Rob. t-test  Rob. p-value  
BETA_CR2_second     1.090448  2.755158e-01  
BETA_CR3_first     -4.668832  3.029174e-06  
BETA_CR3_second     0.607401  5.435846e-01  
BETA_CR4_first      7.449123  9.392487e-14  
BETA_IVT            3.915674  9.015201e-05  
BETA_WT            -9.312022  0.000000e+00  


## Model 015 general model, non-time-critical trips only, ASC
### dep1 = 0
### dep2 = [ASC_second] + [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt

In [16]:
# filter records
df_used_03 = df_used[df_used['time_crit']==0]

In [17]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used_03)

In [18]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
ASC_SECOND = Beta('ASC_SECOND',0,None,None,0)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = ASC_SECOND + \
     BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [19]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_015_general_nontimecrit_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_SECOND=	0.615
BETA_CR2_second=	0.354
BETA_CR3_first=	-0.769
BETA_CR3_second=	0.261
BETA_CR4_first=	1.38
BETA_IVT=	0.0345
BETA_WT=	-0.259
                    Value   Std err     t-test       p-value  Rob. Std err  \
ASC_SECOND       0.614922  0.127561   4.820601  1.431267e-06      0.138543   
BETA_CR2_second  0.354140  0.095648   3.702534  2.134568e-04      0.098974   
BETA_CR3_first  -0.769387  0.106776  -7.205599  5.779821e-13      0.111834   
BETA_CR3_second  0.260781  0.111044   2.348447  1.885189e-02      0.113627   
BETA_CR4_first   1.384309  0.104583  13.236505  0.000000e+00      0.105964   
BETA_IVT         0.034539  0.006588   5.242274  1.586095e-07      0.008563   
BETA_WT         -0.259043  0.027720  -9.344877  0.000000e+00      0.027497   

                 Rob. t-test  Rob. p-value  
ASC_SECOND          4.438482  9.059535e-06  
BETA_CR2_second     3.578132  3.460590e-04  
BETA_CR3_first     -6.879749  5.995870e-12  
BETA_CR3_second     2.295065  2.172941e-02  
BETA_CR4_

## Model 016 general model, non-time-critical trips only
### dep1 = 0
### dep2 = [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt

In [20]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [21]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_016_general_nontimecrit"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_CR2_second=	0.662
BETA_CR3_first=	-0.462
BETA_CR3_second=	0.568
BETA_CR4_first=	1.69
BETA_IVT=	0.0345
BETA_WT=	-0.259
                    Value   Std err     t-test       p-value  Rob. Std err  \
BETA_CR2_second  0.661598  0.140575   4.706363  2.521747e-06      0.150445   
BETA_CR3_first  -0.461931  0.154392  -2.991935  2.772153e-03      0.165957   
BETA_CR3_second  0.568241  0.161635   3.515584  4.387883e-04      0.169895   
BETA_CR4_first   1.691770  0.151359  11.177181  0.000000e+00      0.158066   
BETA_IVT         0.034539  0.006588   5.242275  1.586084e-07      0.008563   
BETA_WT         -0.259043  0.027720  -9.344854  0.000000e+00      0.027497   

                 Rob. t-test  Rob. p-value  
BETA_CR2_second     4.397593      0.000011  
BETA_CR3_first     -2.783440      0.005379  
BETA_CR3_second     3.344650      0.000824  
BETA_CR4_first     10.702931      0.000000  
BETA_IVT            4.033502      0.000055  
BETA_WT            -9.420723      0.000000  


## Model 017 - extended model, ASC
### added Boolean variables: [commuter] - using this route min. 2 days per week; [age_50-65]; and [age_65]
### dep1 = 0
### dep2 = [ASC_second] + [B_timecrit] * s1 + [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt + [B_commuter] * s6 + [B_age_50-65] * s7 + [B_age_65] * s8

In [22]:
df_used_04 = df[['pass_ID','scenario','time_crit','B_cr4first','B_cr3first','B_cr3second','B_cr2second','commuter','age_50_65','age_65','IVT_time','WT_time','output_choice']]
df_used_04.head(20)

,pass_ID,scenario,time_crit,B_cr4first,B_cr3first,B_cr3second,B_cr2second,commuter,age_50_65,age_65,IVT_time,WT_time,output_choice
index,,,,,,,,,,,,,
0,1,1,1,0,1,0,1,0,0,1,20,10,0
1,1,2,1,0,1,0,1,0,0,1,20,5,1
2,1,3,1,1,0,1,0,0,0,1,20,10,0
3,1,4,1,1,0,1,0,0,0,1,20,5,1
4,1,5,1,1,0,0,1,0,0,1,20,10,0
5,1,6,1,1,0,0,1,0,0,1,20,5,1
6,2,1,1,0,1,0,1,1,0,0,15,10,0
7,2,2,1,0,1,0,1,1,0,0,15,5,0
8,2,3,1,1,0,1,0,1,0,0,15,10,0


In [23]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used_04)

In [24]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
ASC_SECOND = Beta('ASC_SECOND',0,None,None,0)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)
BETA_commuter = Beta('BETA_commuter',0, None,None,0)
BETA_AGE_50_65 = Beta('BETA_AGE_50_65',0,None,None,0)
BETA_AGE_65 = Beta('BETA_AGE_65',0,None,None,0)

# variables
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
s6 = commuter
s7 = age_50_65
s8 = age_65
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = ASC_SECOND + \
     BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt + \
     BETA_commuter * s6 + \
     BETA_AGE_50_65 * s7 + \
     BETA_AGE_65 * s8

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [25]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_017_extended_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_SECOND=	0.426
BETA_AGE_50_65=	0.936
BETA_AGE_65=	2.2
BETA_CR2_second=	0.248
BETA_CR3_first=	-0.818
BETA_CR3_second=	0.178
BETA_CR4_first=	1.24
BETA_IVT=	0.0186
BETA_WT=	-0.263
BETA_commuter=	-0.174
                    Value   Std err     t-test   p-value  Rob. Std err  \
ASC_SECOND       0.425641  0.105888   4.019717  0.000058      0.105078   
BETA_AGE_50_65   0.935683  0.231527   4.041354  0.000053      0.246502   
BETA_AGE_65      2.201935  0.222559   9.893717  0.000000      0.235762   
BETA_CR2_second  0.247966  0.072332   3.428171  0.000608      0.072403   
BETA_CR3_first  -0.817968  0.088229  -9.270954  0.000000      0.088104   
BETA_CR3_second  0.177674  0.082241   2.160400  0.030742      0.082004   
BETA_CR4_first   1.243609  0.080959  15.360926  0.000000      0.079250   
BETA_IVT         0.018565  0.004178   4.444031  0.000009      0.004375   
BETA_WT         -0.262801  0.020157 -13.038007  0.000000      0.020134   
BETA_commuter   -0.174100  0.134787  -1.291672  0.196471  

## Model 018 - extended model
### added Boolean variables: [commuter] - using this route min. 2 days per week; [age_50-65]; and [age_65]
### dep1 = 0
### dep2 = [B_timecrit] * s1 + [B_cr4_first] * s2 + [B_cr3_first] * s3 + [B_cr3_second] * s4 + [B_cr2_second] * s5  + [B_ivt] * ivt + [B_wt] * wt + [B_commuter] * s6 + [B_age_50-65] * s7 + [B_age_65] * s8

In [26]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_CR4_first = Beta('BETA_CR4_first',0,None,None,0)
BETA_CR3_first = Beta('BETA_CR3_first',0,None,None,0)
BETA_CR3_second = Beta('BETA_CR3_second',0,None,None,0)
BETA_CR2_second = Beta('BETA_CR2_second',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)
BETA_commuter = Beta('BETA_commuter',0, None,None,0)
BETA_AGE_50_65 = Beta('BETA_AGE_50_65',0,None,None,0)
BETA_AGE_65 = Beta('BETA_AGE_65',0,None,None,0)

# variables
s2 = B_cr4first 
s3 = B_cr3first 
s4 = B_cr3second 
s5 = B_cr2second 
s6 = commuter
s7 = age_50_65
s8 = age_65
ivt = IVT_time 
wt = WT_time 

# utilities
V1 = ASC_FIRST

V2 = BETA_CR4_first * s2 + \
     BETA_CR3_first * s3 + \
     BETA_CR3_second * s4 + \
     BETA_CR2_second * s5 + \
     BETA_IVT * ivt + \
     BETA_WT * wt + \
     BETA_commuter * s6 + \
     BETA_AGE_50_65 * s7 + \
     BETA_AGE_65 * s8

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [27]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_018_extended"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_AGE_50_65=	0.936
BETA_AGE_65=	2.2
BETA_CR2_second=	0.461
BETA_CR3_first=	-0.605
BETA_CR3_second=	0.391
BETA_CR4_first=	1.46
BETA_IVT=	0.0186
BETA_WT=	-0.263
BETA_commuter=	-0.174
                    Value   Std err     t-test   p-value  Rob. Std err  \
BETA_AGE_50_65   0.935708  0.231528   4.041448  0.000053      0.246503   
BETA_AGE_65      2.202016  0.222563   9.893914  0.000000      0.235770   
BETA_CR2_second  0.460873  0.113470   4.061646  0.000049      0.113058   
BETA_CR3_first  -0.605040  0.129648  -4.666805  0.000003      0.129553   
BETA_CR3_second  0.390595  0.126249   3.093838  0.001976      0.125487   
BETA_CR4_first   1.456508  0.119787  12.159165  0.000000      0.117564   
BETA_IVT         0.018564  0.004178   4.443618  0.000009      0.004375   
BETA_WT         -0.262816  0.020157 -13.038681  0.000000      0.020135   
BETA_commuter   -0.174127  0.134787  -1.291864  0.196404      0.131454   

                 Rob. t-test  Rob. p-value  
BETA_AGE_50_65      3.795930  

# models 02x - temporal framework
### (consistent e.g. with the BusMezzo utility/choice model)
### a) ASC / no ASC
### b) 3 scenarios
### c) time-criticality / or not
### = 12 models in total

In [7]:
df_temporal = df[['pass_ID','scenario','time_crit','now_ivt_4','now_ivt_3','wait_ivt_3','wait_ivt_2','WT_time','output_choice']]
df_temporal.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
0,1,1,1,0,20,0,20,10,0
1,1,2,1,0,20,0,20,5,1
2,1,3,1,20,0,20,0,10,0
3,1,4,1,20,0,20,0,5,1
4,1,5,1,20,0,0,20,10,0
5,1,6,1,20,0,0,20,5,1
6,2,1,1,0,15,0,15,10,0
7,2,2,1,0,15,0,15,5,0
8,2,3,1,15,0,15,0,10,0


# Model 021 - time-critical trips, ASC, scenario 01
### dep1 = 0 + ivt_3 * [B_ivt3_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [8]:
# filter records
df_temporal_sc021 = df_temporal[((df_temporal['scenario'] == 1) | (df_temporal['scenario'] == 2)) & (df_temporal['time_crit'] == 1)]
df_temporal_sc021.describe()

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
count,342.000000,342.000000,342.0,342.0,342.000000,342.0,342.000000,342.000000,342.000000
mean,196.824561,1.500000,1.0,0.0,22.321637,0.0,22.321637,7.500000,0.099415
std,110.805389,0.500733,0.0,0.0,12.999280,0.0,12.999280,2.503663,0.299657
min,1.000000,1.000000,1.0,0.0,2.000000,0.0,2.000000,5.000000,0.000000
25%,106.500000,1.000000,1.0,0.0,15.000000,0.0,15.000000,5.000000,0.000000
50%,208.000000,1.500000,1.0,0.0,20.000000,0.0,20.000000,7.500000,0.000000
75%,288.250000,2.000000,1.0,0.0,30.000000,0.0,30.000000,10.000000,0.000000
max,386.000000,2.000000,1.0,0.0,90.000000,0.0,90.000000,10.000000,1.000000


In [9]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_temporal_sc021)

In [10]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_3 = now_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_3_now * NOW_IVT_3

V2 = ASC_WAIT + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [11]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_021_scenario_01_timecrit_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	-0.929
BETA_IVT_2_WAIT=	0.0159
BETA_IVT_3_NOW=	-0.0159
BETA_WT_WAIT=	-0.303
                    Value   Std err    t-test   p-value  Rob. Std err  \
ASC_WAIT        -0.928565  0.644359 -1.441068  0.149565      0.614852   
BETA_IVT_2_WAIT  0.015930  0.006224  2.559598  0.010479      0.005628   
BETA_IVT_3_NOW  -0.015930  0.006224 -2.559598  0.010479      0.005628   
BETA_WT_WAIT    -0.302763  0.088890 -3.406040  0.000659      0.090150   

                 Rob. t-test  Rob. p-value  
ASC_WAIT           -1.510226      0.130986  
BETA_IVT_2_WAIT     2.830512      0.004647  
BETA_IVT_3_NOW     -2.830512      0.004647  
BETA_WT_WAIT       -3.358434      0.000784  


# Model 022 - time-critical trips, scenario 01
### dep1 = 0 + ivt_3 * [B_ivt3_now] 
### dep2 = ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [12]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_3 = now_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_3_now * NOW_IVT_3

V2 = B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [13]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_022_scenario_01_timecrit"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_IVT_2_WAIT=	0.0121
BETA_IVT_3_NOW=	-0.0121
BETA_WT_WAIT=	-0.413
                    Value   Std err    t-test       p-value  Rob. Std err  \
BETA_IVT_2_WAIT  0.012062  0.005740  2.101418  3.560433e-02      0.005752   
BETA_IVT_3_NOW  -0.012062  0.005740 -2.101418  3.560433e-02      0.005752   
BETA_WT_WAIT    -0.412756  0.054839 -7.526722  5.195844e-14      0.059258   

                 Rob. t-test  Rob. p-value  
BETA_IVT_2_WAIT     2.096902  3.600223e-02  
BETA_IVT_3_NOW     -2.096902  3.600223e-02  
BETA_WT_WAIT       -6.965370  3.275380e-12  


# Model 023 - non-time-critical trips, ASC, scenario 01
### dep1 = 0 + ivt_3 * [B_ivt3_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [14]:
# filter records
df_temporal_sc023 = df_temporal[((df_temporal['scenario'] == 1) | (df_temporal['scenario'] == 2)) & (df_temporal['time_crit'] == 0)]

In [15]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_temporal_sc023)

In [16]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_3 = now_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_3_now * NOW_IVT_3

V2 = ASC_WAIT + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [17]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_023_scenario_01_nontimecrit_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	0.17
BETA_IVT_2_WAIT=	0.0145
BETA_IVT_3_NOW=	-0.0145
BETA_WT_WAIT=	-0.237
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         0.169961  0.398432  0.426575  6.696893e-01      0.411924   
BETA_IVT_2_WAIT  0.014550  0.005174  2.812063  4.922489e-03      0.005692   
BETA_IVT_3_NOW  -0.014550  0.005174 -2.812063  4.922489e-03      0.005692   
BETA_WT_WAIT    -0.236733  0.046489 -5.092283  3.537782e-07      0.046361   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            0.412603  6.798975e-01  
BETA_IVT_2_WAIT     2.556243  1.058091e-02  
BETA_IVT_3_NOW     -2.556243  1.058091e-02  
BETA_WT_WAIT       -5.106343  3.284537e-07  


# Model 024 - non-time-critical trips, scenario 01
### dep1 = 0 + ivt_3 * [B_ivt3_now] 
### dep2 = ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [18]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_3 = now_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_3_now * NOW_IVT_3

V2 = ASC_WAIT + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [19]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_024_scenario_01_nontimecrit"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	0.17
BETA_IVT_2_WAIT=	0.0145
BETA_IVT_3_NOW=	-0.0145
BETA_WT_WAIT=	-0.237
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         0.169961  0.398432  0.426575  6.696893e-01      0.411924   
BETA_IVT_2_WAIT  0.014550  0.005174  2.812063  4.922489e-03      0.005692   
BETA_IVT_3_NOW  -0.014550  0.005174 -2.812063  4.922489e-03      0.005692   
BETA_WT_WAIT    -0.236733  0.046489 -5.092283  3.537782e-07      0.046361   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            0.412603  6.798975e-01  
BETA_IVT_2_WAIT     2.556243  1.058091e-02  
BETA_IVT_3_NOW     -2.556243  1.058091e-02  
BETA_WT_WAIT       -5.106343  3.284537e-07  


# Model 025 - time-critical trips, ASC, scenario 02
### dep1 = 0 + ivt_4 * [B_ivt4_now] 
### dep2 = [ASC_wait] + ivt_3 * [B_ivt3_wait] + wt * [B_WT_wait]

In [20]:
# filter records
df_temporal_sc025 = df_temporal[((df_temporal['scenario'] == 3) | (df_temporal['scenario'] == 4)) & (df_temporal['time_crit'] == 1)]
df_temporal_sc025.describe()

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
count,342.000000,342.000000,342.0,342.000000,342.0,342.000000,342.0,342.000000,342.000000
mean,196.824561,3.500000,1.0,22.321637,0.0,22.321637,0.0,7.500000,0.394737
std,110.805389,0.500733,0.0,12.999280,0.0,12.999280,0.0,2.503663,0.489510
min,1.000000,3.000000,1.0,2.000000,0.0,2.000000,0.0,5.000000,0.000000
25%,106.500000,3.000000,1.0,15.000000,0.0,15.000000,0.0,5.000000,0.000000
50%,208.000000,3.500000,1.0,20.000000,0.0,20.000000,0.0,7.500000,0.000000
75%,288.250000,4.000000,1.0,30.000000,0.0,30.000000,0.0,10.000000,1.000000
max,386.000000,4.000000,1.0,90.000000,0.0,90.000000,0.0,10.000000,1.000000


In [21]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_temporal_sc025)

In [22]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_3 = wait_ivt_3
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = ASC_WAIT + \
     B_IVT_3_wait * WAIT_IVT_3 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [23]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_025_scenario_02_timecrit_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	1.25
BETA_IVT_3_WAIT=	0.00905
BETA_IVT_4_NOW=	-0.00905
BETA_WT_WAIT=	-0.286
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         1.251089  0.401908  3.112876  1.852740e-03      0.398442   
BETA_IVT_3_WAIT  0.009049  0.004558  1.985551  4.708320e-02      0.004468   
BETA_IVT_4_NOW  -0.009049  0.004558 -1.985551  4.708320e-02      0.004468   
BETA_WT_WAIT    -0.285540  0.047885 -5.963060  2.475582e-09      0.047956   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            3.139955  1.689740e-03  
BETA_IVT_3_WAIT     2.025540  4.281197e-02  
BETA_IVT_4_NOW     -2.025540  4.281197e-02  
BETA_WT_WAIT       -5.954204  2.613414e-09  


# Model 026 - time-critical trips, scenario 02
### dep1 = ivt_4 * [B_ivt4_now] 
### dep2 = ivt_3 * [B_ivt3_wait] + wt * [B_WT_wait]

In [24]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_3 = wait_ivt_3
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = B_IVT_3_wait * WAIT_IVT_3 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [25]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_026_scenario_02_timecrit"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_IVT_3_WAIT=	0.0155
BETA_IVT_4_NOW=	-0.0155
BETA_WT_WAIT=	-0.167
                    Value   Std err    t-test       p-value  Rob. Std err  \
BETA_IVT_3_WAIT  0.015519  0.004092  3.792477  1.491522e-04      0.004161   
BETA_IVT_4_NOW  -0.015519  0.004092 -3.792477  1.491522e-04      0.004161   
BETA_WT_WAIT    -0.166968  0.028025 -5.957801  2.556548e-09      0.027871   

                 Rob. t-test  Rob. p-value  
BETA_IVT_3_WAIT     3.729851  1.915930e-04  
BETA_IVT_4_NOW     -3.729851  1.915930e-04  
BETA_WT_WAIT       -5.990644  2.090118e-09  


# Model 027 - non-time-critical trips, ASC, scenario 02
### dep1 = 0 + ivt_4 * [B_ivt4_now] 
### dep2 = [ASC_wait] + ivt_3 * [B_ivt3_wait] + wt * [B_WT_wait]

In [26]:
# filter records
df_temporal_sc027 = df_temporal[((df_temporal['scenario'] == 3) | (df_temporal['scenario'] == 4)) & (df_temporal['time_crit'] == 0)]
df_temporal_sc027.describe()

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
count,418.000000,418.000000,418.0,418.000000,418.0,418.000000,418.0,418.000000,418.000000
mean,190.430622,3.500000,0.0,21.641148,0.0,21.641148,0.0,7.500000,0.720096
std,111.982211,0.500599,0.0,10.835169,0.0,10.835169,0.0,2.502996,0.449490
min,5.000000,3.000000,0.0,2.000000,0.0,2.000000,0.0,5.000000,0.000000
25%,93.000000,3.000000,0.0,15.000000,0.0,15.000000,0.0,5.000000,0.000000
50%,185.000000,3.500000,0.0,20.000000,0.0,20.000000,0.0,7.500000,1.000000
75%,290.000000,4.000000,0.0,30.000000,0.0,30.000000,0.0,10.000000,1.000000
max,389.000000,4.000000,0.0,85.000000,0.0,85.000000,0.0,10.000000,1.000000


In [27]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_temporal_sc027)

In [28]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_3 = wait_ivt_3
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = ASC_WAIT + \
     B_IVT_3_wait * WAIT_IVT_3 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [29]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_027_scenario_02_nontimecrit_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	2.27
BETA_IVT_3_WAIT=	0.0175
BETA_IVT_4_NOW=	-0.0175
BETA_WT_WAIT=	-0.261
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         2.269397  0.458124  4.953677  7.282411e-07      0.506188   
BETA_IVT_3_WAIT  0.017458  0.005909  2.954660  3.130143e-03      0.008689   
BETA_IVT_4_NOW  -0.017458  0.005909 -2.954660  3.130143e-03      0.008689   
BETA_WT_WAIT    -0.261114  0.047904 -5.450752  5.015715e-08      0.047609   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            4.483309  7.349439e-06  
BETA_IVT_3_WAIT     2.009177  4.451842e-02  
BETA_IVT_4_NOW     -2.009177  4.451842e-02  
BETA_WT_WAIT       -5.484591  4.144279e-08  


# Model 028 - non-time-critical trips, scenario 02
### dep1 = ivt_4 * [B_ivt4_now] 
### dep2 = ivt_3 * [B_ivt3_wait] + wt * [B_WT_wait]

In [30]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_3 = wait_ivt_3
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = B_IVT_3_wait * WAIT_IVT_3 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [31]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_028_scenario_02_nontimecrit"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_IVT_3_WAIT=	0.033
BETA_IVT_4_NOW=	-0.033
BETA_WT_WAIT=	-0.0702
                    Value   Std err    t-test       p-value  Rob. Std err  \
BETA_IVT_3_WAIT  0.033025  0.005273  6.263566  3.762728e-10      0.007548   
BETA_IVT_4_NOW  -0.033025  0.005273 -6.263566  3.762728e-10      0.007548   
BETA_WT_WAIT    -0.070179  0.027545 -2.547800  1.084047e-02      0.035392   

                 Rob. t-test  Rob. p-value  
BETA_IVT_3_WAIT     4.375389      0.000012  
BETA_IVT_4_NOW     -4.375389      0.000012  
BETA_WT_WAIT       -1.982865      0.047383  


# Model 029 - time-critical trips, ASC, scenario 03
### dep1 = 0 + ivt_4 * [B_ivt4_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [32]:
# filter records
df_temporal_sc029 = df_temporal[((df_temporal['scenario'] == 5) | (df_temporal['scenario'] == 6)) & (df_temporal['time_crit'] == 1)]
df_temporal_sc029.describe()

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
count,342.000000,342.000000,342.0,342.000000,342.0,342.0,342.000000,342.000000,342.000000
mean,196.824561,5.500000,1.0,22.321637,0.0,0.0,22.321637,7.500000,0.406433
std,110.805389,0.500733,0.0,12.999280,0.0,0.0,12.999280,2.503663,0.491887
min,1.000000,5.000000,1.0,2.000000,0.0,0.0,2.000000,5.000000,0.000000
25%,106.500000,5.000000,1.0,15.000000,0.0,0.0,15.000000,5.000000,0.000000
50%,208.000000,5.500000,1.0,20.000000,0.0,0.0,20.000000,7.500000,0.000000
75%,288.250000,6.000000,1.0,30.000000,0.0,0.0,30.000000,10.000000,1.000000
max,386.000000,6.000000,1.0,90.000000,0.0,0.0,90.000000,10.000000,1.000000


In [33]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_temporal_sc029)

In [34]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = ASC_WAIT + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [35]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_029_scenario_03_timecrit_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	1.37
BETA_IVT_2_WAIT=	0.011
BETA_IVT_4_NOW=	-0.011
BETA_WT_WAIT=	-0.307
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         1.370346  0.402308  3.406208  6.587188e-04      0.399427   
BETA_IVT_2_WAIT  0.010963  0.004636  2.364894  1.803524e-02      0.004603   
BETA_IVT_4_NOW  -0.010963  0.004636 -2.364894  1.803524e-02      0.004603   
BETA_WT_WAIT    -0.306627  0.048233 -6.357230  2.054243e-10      0.048342   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            3.430780  6.018474e-04  
BETA_IVT_2_WAIT     2.381512  1.724171e-02  
BETA_IVT_4_NOW     -2.381512  1.724171e-02  
BETA_WT_WAIT       -6.342859  2.255394e-10  


# Model 030 - time-critical trips, scenario 03
### dep1 = ivt_4 * [B_ivt4_now] 
### dep2 = ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [36]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [37]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_030_scenario_03_timecrit"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_IVT_2_WAIT=	0.0181
BETA_IVT_4_NOW=	-0.0181
BETA_WT_WAIT=	-0.177
                    Value   Std err    t-test       p-value  Rob. Std err  \
BETA_IVT_2_WAIT  0.018078  0.004201  4.303138  1.683958e-05      0.004368   
BETA_IVT_4_NOW  -0.018078  0.004201 -4.303138  1.683958e-05      0.004368   
BETA_WT_WAIT    -0.177080  0.028540 -6.204682  5.480760e-10      0.028714   

                 Rob. t-test  Rob. p-value  
BETA_IVT_2_WAIT     4.138406  3.497270e-05  
BETA_IVT_4_NOW     -4.138406  3.497270e-05  
BETA_WT_WAIT       -6.167037  6.958134e-10  


# Model 031 - non-time-critical trips, ASC, scenario 03
### dep1 = 0 + ivt_4 * [B_ivt4_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [38]:
# filter records
df_temporal_sc031 = df_temporal[((df_temporal['scenario'] == 5) | (df_temporal['scenario'] == 6)) & (df_temporal['time_crit'] == 0)]
df_temporal_sc031.describe()

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
count,418.000000,418.000000,418.0,418.000000,418.0,418.0,418.000000,418.000000,418.000000
mean,190.430622,5.500000,0.0,21.641148,0.0,0.0,21.641148,7.500000,0.736842
std,111.982211,0.500599,0.0,10.835169,0.0,0.0,10.835169,2.502996,0.440875
min,5.000000,5.000000,0.0,2.000000,0.0,0.0,2.000000,5.000000,0.000000
25%,93.000000,5.000000,0.0,15.000000,0.0,0.0,15.000000,5.000000,0.000000
50%,185.000000,5.500000,0.0,20.000000,0.0,0.0,20.000000,7.500000,1.000000
75%,290.000000,6.000000,0.0,30.000000,0.0,0.0,30.000000,10.000000,1.000000
max,389.000000,6.000000,0.0,85.000000,0.0,0.0,85.000000,10.000000,1.000000


In [39]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_temporal_sc031)

In [40]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = ASC_WAIT + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [41]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_031_scenario_03_nontimecrit_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	2.4
BETA_IVT_2_WAIT=	0.0207
BETA_IVT_4_NOW=	-0.0207
BETA_WT_WAIT=	-0.282
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         2.402979  0.476982  5.037883  4.707080e-07      0.528097   
BETA_IVT_2_WAIT  0.020739  0.006199  3.345541  8.212229e-04      0.008331   
BETA_IVT_4_NOW  -0.020739  0.006199 -3.345541  8.212228e-04      0.008331   
BETA_WT_WAIT    -0.281743  0.049909 -5.645186  1.650030e-08      0.048936   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            4.550259  5.357984e-06  
BETA_IVT_2_WAIT     2.489383  1.279649e-02  
BETA_IVT_4_NOW     -2.489383  1.279649e-02  
BETA_WT_WAIT       -5.757391  8.542399e-09  


# Model 032 - non-time-critical trips, scenario 03
### dep1 = ivt_4 * [B_ivt4_now] 
### dep2 = ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [42]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [43]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_777_032_scenario_03_nontimecrit"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_IVT_2_WAIT=	0.0371
BETA_IVT_4_NOW=	-0.0371
BETA_WT_WAIT=	-0.0789
                    Value   Std err    t-test       p-value  Rob. Std err  \
BETA_IVT_2_WAIT  0.037130  0.005554  6.685394  2.303024e-11      0.007123   
BETA_IVT_4_NOW  -0.037130  0.005554 -6.685394  2.303024e-11      0.007123   
BETA_WT_WAIT    -0.078915  0.028273 -2.791198  5.251336e-03      0.032791   

                 Rob. t-test  Rob. p-value  
BETA_IVT_2_WAIT     5.212789  1.860226e-07  
BETA_IVT_4_NOW     -5.212789  1.860226e-07  
BETA_WT_WAIT       -2.406602  1.610171e-02  
